# Bibliotecas

In [ ]:
# Ignorando alguns logs
import warnings
warnings.simplefilter("ignore")

# Processamento dos dados
import glob
import pandas as pd
import numpy as np

# Divisão dos dados
from sklearn.model_selection import train_test_split

# Importando os frameworks
import torch
import flash
from pytorch_lightning import seed_everything
from flash.image import ImageClassificationData, ImageClassifier

# Leitura dos dados

Para começar o experimento, é necessario carregar os dados utilizados. O Pytorch funciona de forma lazy, ou seja, não vamos carregar todas as fotos em disco, devemos apenas ter um Dataframe que possui o caminho até a imagem e sua classe.

## Processando os caminhos
Função para dar um parse do caminho e obter o dicionario com a chave para o caminho absoluto e a classe da imagem

In [ ]:
def class_from_path(path):

    list_ = path.split('/')
    class_ = list_[-2]

    return {
        'class': class_,
        'path': path
    }

String de busca para obter os caminhos das classes aloevera e cassava

In [ ]:
data_paths_fruits = [
                     'data/new_types_fruits/aloevera/*',
                     'data/new_types_fruits/cassava/*'
                     ]

## Mostrando o Dataframe

In [ ]:
fruits_paths = [glob.glob(path) for path in data_paths_fruits]

In [ ]:
fruits_paths

In [ ]:
paths = [*fruits_paths[0], *fruits_paths[1]]

In [ ]:
classes = map(class_from_path, paths)
df_paths = pd.DataFrame(classes)

df_paths['class'] = df_paths['class'] == 'aloevera'

df_paths.head()

# Separando em Treino, Teste e Validação

In [ ]:
df_, test = train_test_split(df_paths, test_size=0.3, random_state=42)

train, val = train_test_split(df_, test_size=0.2, random_state=42)

train.shape[0], val.shape[0], test.shape[0]

# Seed
Para tornar o experimento reprodutivel, deve-se travar a seed dos frameworks

In [ ]:
seed_everything(42)

# Modulo Dataset
Com o Dataframe gerado, devemos instaciar uma classe de dados que permite acessar as imagens quando necessario.

In [ ]:
datamodule_fruits = ImageClassificationData.from_data_frame(
     "path",
     "class",
     train_data_frame=train,
    
     val_data_frame = val,

     test_data_frame = test,

     transform_kwargs=dict(image_size=(128, 128)),

     batch_size=2

     )

# Montando o modelo
Agora começam as configurações utilizadas pelo treinamento

In [ ]:
model = ImageClassifier(
    # Rede utilizada, poderiamos ter uma classe personalizada, entretanto, 
    # usamos uma rede pronta para facilitar
    backbone="resnet18",

    # defindo a meta-heuristica, aqui podemos utilizar as seguintes abordagens:
    # maml, anil, metaoptnet e prototypicalnetworks    
    training_strategy="maml",

    # Este atributo é referente a rede, no caso, não vamos utilizar uma rede 
    # pretreinada
    pretrained=False,

    # Agora, estes são atributos referentes à meta-heuristica
    training_strategy_kwargs={
        # Quantas epocas serão utilizadas
        "epoch_length": 50,
        
        # tamanho do batch
        "meta_batch_size": 2,
        
        # Quantas tarefas de treino
        "num_tasks": 50,
        
        # Referente aos testes finais
        "test_num_tasks": 50,
        
        # Nossos ways e shots
        "ways": datamodule_fruits.num_classes,
        "shots": 2,

        # Ways e Shots no teste
        "test_ways": 2,
        "test_shots": 1,

        # atributo para inserir os resultados do teste no tensorboard 
        # "test_queries": 15,
    },

    # Finalmente, nosso otimizador Pytorch
    optimizer=torch.optim.Adam,
    learning_rate=0.001,
)


# Tensorboard
Se quiser acompanhar os resultados pelo Tensorboard, é so instaciar este objeto com o path desejado para guardar os logs

In [ ]:
# from pytorch_lightning import loggers as pl_loggers

# tb_logger = pl_loggers.TensorBoardLogger(save_dir="/content/drive/MyDrive/HerbaData/savemodel/meta_models/logs/")

# Trainer
Worker que vai processar o treinamento

In [ ]:
trainer = flash.Trainer(
    max_epochs=50,
    # precision=16,
    # accelerator="cpu",
    # gpus=int(torch.cuda.is_available()),
    # logger=tb_logger,
    # tpu_cores=[5]
)

# Treinando o modelo!

In [ ]:
trainer.fit(model, datamodule=datamodule_fruits)

# Agora é só salvar

In [ ]:
trainer.save_checkpoint("/content/drive/MyDrive/HerbaData/savemodel/meta_models/image_classification_model_fruits_maml.pt")

# Teste 
Se vc quiser realizar os testes so ao final, pode usar este comando para gerar um report resumido dos resultados

In [ ]:
trainer.test(model, datamodule=datamodule_fruits)